In [1]:
import pandas as pd

In [14]:
dlc = pd.read_json('../../data/domain-link-counts.json/part-00000-92c7e08d-cf21-44b8-a362-8dbfc7dae410-c000.json', lines=True)

In [15]:
dlc

,count,domain
0,13798511,nytimes.com
1,12079379,cnn.com
2,10208267,washingtonpost.com
3,8780829,thehill.com
4,8011778,theguardian.com
5,6838388,foxnews.com
6,6372195,bbc.co.uk
7,4581144,reuters.com
8,4380006,breitbart.com
9,4061263,huffingtonpost.com
